在前面的部分我们已经介绍了微型app(一个py文件)和小型app(一个多py文件组成的单模块程序),现在我们来看看如何组织中型应用

## 首先是文件组织结构



        msgboard\
                |--app\
                |     |--__init__.py
                |     |--static\
                |     |        |--js\
                |     |        |--css\
                |     |        |--img\
                |     |        |--fonts\
                |     |     
                |     |--templates\
                |     |           |--base.html
                |     |           |--blocks\      
                |     |           |--index.html
                |     |--model.py
                |     |--error.py
                |     |--main\
                |     |      |--__init__.py
                |     |      |--view.py
                |     |      |--form.py
                |     |      |--static\
                |     |      |         |--js\
                |     |      |         |--css\
                |     |      |         |--img\
                |     |      |
                |     |      |--templates\
                |     |
                |     
                |--env\
                |--config.py
                |--manager.py
                |--test\
                |--License
                |--requirement.txt
            
            
           
                        
应对这种规模的代码我们已经不能靠jupyter来做编写工作啦,这个人时候是atom出场的时候啦                        

这种结构有 4 个顶级文件夹:

+ Flask 程序一般都保存在名为 app 的包中;
+ 和之前一样,migrations 文件夹包含数据库迁移脚本;
+ 单元测试编写在 test 包中;
+ 和之前一样,venv 文件夹包含 Python 虚拟环境。

> 工厂方法

在单个文件中开发程序很方便,但却有个很大的缺点,因为程序在全局作用域中创建,所 以无法动态修改配置。
运行脚本时,程序实例已经创建,再修改配置为时已晚。这一点对单元测试尤其重要,因为有时为了提高测试覆盖度,必须在不同的配置环境中运行程序。
这个问题的解决方法是延迟创建程序实例,把创建过程移到可显式调用的工厂函数中。这种方法不仅可以给脚本留出配置程序的时间,还能够创建多个程序实例,这些实例有时在 测试中非常有用。

程序的工厂函数在 app 包的构造文件中定义,
构造文件导入了大多数正在使用的 Flask 扩展。由于尚未初始化所需的程序实例,所以没 有初始化扩展,创建扩展类时没有向构造函数传入参数。create_app() 函数就是程序的工 厂函数,接受一个参数,是程序使用的配置名。配置类在 config.py 文件中定义,其中保存 的配置可以使用 Flask app.config 配置对象提供的 from_object() 方法直接导入程序。至 于配置对象,则可以通过名字从 config 字典中选择。程序创建并配置好后,就能初始化 扩展了。在之前创建的扩展对象上调用 init_app() 可以完成初始化过程。

`app/__init__.py`:

```python
from flask import Flask
from config import config
from flask_mail import Mail
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()
appmail = Mail()
def create_app(config_name):
    app = Flask(__name__)
    app.config.from_object(config[config_name])
    config[config_name].init_app(app)
    appmail.init_app(app)
    db.init_app(app)
    from .Main import main as main_blueprint
    app.register_blueprint(main_blueprint)
    from .Login import login as login_blueprint
    app.register_blueprint(login_blueprint)

    return app
```

这个部分我们就讲讲怎么用蓝图吧

> 蓝图的注册

flask通过蓝图进行模块化,蓝图可以先生成再注册,只有注册到的蓝图才会被使用

+ 定义蓝图,在app目录下的子模块中,我们一般在`__init__.py`中定义蓝图

`app/Main/__init__.py`:

```python
from flask import Blueprint
main = Blueprint('main', __name__,static_folder='static', static_url_path='/main',template_folder='templates')
from . import view
```

其中要注意的是`static_folder='static'和template_folder='templates'`是将子模块中的对应文件夹也注册到静态文件夹或者模板文件夹中,在使用模板的时候,框架会先找默认的app下的模板,如果没有才去子模块中查找.因此要注意.

而如果要用`url_for()`函数,则需要在其中注明所属的蓝图,比如:

`url_for("main.index",filename=xxxxx)`

需要注意的是静态文件必须注册 `static_url_path='/main'`才能被找到

在其中也可以增加一个`url_prefix开始x='/login'`,这样改蓝图中的所有view都将自动以对应的url_prefix开始

定义好蓝图后就得在`app/__init__.py`下注册了,就如上面所见

    
        from .Main import main as main_blueprint
        app.register_blueprint(main_blueprint)
        from .Login import login as login_blueprint
        app.register_blueprint(login_blueprint)

+ 再定义view式我们要把`app.route()`修饰符换成`model.route()`


```python

@main.route('/')
def index():
    img_url = "img/index/carousel"
    return render_template("index.html", carousel=set_carousel(img_url))
    
```